#### Задача 1 (5 баллов).

Напишите программу, которая будет уметь читать содержимое файла .conllu и выводить по нему статистику. Что должно быть: 

- класс Token для хранения информации по токенам
- класс Sentence, в котором хранится список токенов и исходный текст предложения
- класс Reader, в котором хранится список предложений, а также есть методы "прочитать", "вывести статистику по частям речи" (выводит процентное соотношение частей речи в файле) и "искать вхождения слова по лемме" (выводит отдельные токены, у которых лемма совпала с искомой). 

Бонусы для джедаев (если их выполнить, можно получить доп. балл):

1) если переопределите магический метод \_\_getitem\_\_ у класса Sentence (он принимает self и индекс), то сможете брать срезы по токенам, как у обычного списка. 

        def __getitem__(self, index):
            ...
            return <тут конкретный токен с таким индексом>

2) если переопределите магический метод \_\_eq\_\_ у токена, то можно будет использовать оператор сравнения == с экземплярами класса Token. Причем необязательно сравнивать только токены с токенами: можно сравнивать токен со строкой, если в этом методе где-то использовать функцию isinstance(obj, type), которая проверяет, что объект obj принадлежит к классу type. Тогда можно проверять как 'лемма' == token.

Для тестирования можно взять любой .conllu, они в большом количестве водятся [тут](https://github.com/UniversalDependencies/). Чтобы загрузить файл с гитхаба в колаб, нужно открыть в гитхабе его "сырую" версию по кнопочке raw (вверху справа), скопировать адрес из адресной строки браузера и в колабе вписать:

    !wget <address>

In [35]:
class Token: #для ридера нужны только части речи (4 позиция), лемма (3 позиция)
    def __init__(self, line):
        info = line.split('\t')
        self.token_id = info[0]
        self.form = info[1]
        self.lemma = info[2]
        self.upos = info[3]

class Sentence:
    def __init__(self, lines):
        self.token_list = []
        self.text = ''
        for line in lines:
            if line.startswith('# text'):
                self.text = line[line.index('=') + 2:] #отрезаем предложение
            elif line[0].isdigit() and '-' not in line.split('\t')[0]:
                token = Token(line)
                self.token_list.append(token)

class Reader:
    def __init__(self):
        self.sentences = []
    def read(self, file_path):
        with open(file_path, 'r', encoding='utf8') as file:
            sentence_inf = []
            for line in file:
                if line.startswith('# text') or line[0].isdigit():
                    sentence_inf.append(line)
                elif not line.strip():
                    self.sentences.append(Sentence(sentence_inf))
                    sentence_inf = []
            if sentence_inf:
                self.sentences.append(Sentence(sentence_inf))
    
    def upos_statistics(self):
        all_upos = {} #словарь с подсчетом частей речи
        total = 0 #общее кол-во
        for sentence in self.sentences:
            for token in sentence.token_list:
                upos = token.upos
                if upos != '_':
                    all_upos[upos] = all_upos.get(upos, 0) + 1
                    total += 1
        print('Статистика частей речи')
        for upos, amount in all_upos.items():
            percentage = (amount / total) * 100
            print(f'{upos:<{max(len(x) for x in all_upos.keys())}}: {round(percentage, 2)}%')

    def find_by_lemma(self, lemma):
        res = []
        for sentence in self.sentences:
            for token in sentence.token_list:
                if token.lemma == lemma:
                    data = [token, sentence.text]
                    res.append(data)
        return res

In [ ]:
reader = Reader()

while True:
    print('Выберите, что вы хотите сделать\n\t0 - выйти\n\t1 - прочитать файл\n\t2 - вывести статистику по частям речи\n\t3 - искать вхождения по лемме')
    user = int(input())
    if user == 0: #exit
        break
    elif user == 1: #reading
        # user_path = input('Введите путь в файлу')
        user_path = 'fr_fqb-ud-test.conllu'
        reader.read(user_path)
    elif user == 2: #statistics
        reader.upos_statistics()
    elif user == 3: #search by lemma
        user_lemma = input('Введите лемму')
        result = reader.find_by_lemma(user_lemma)
        print(f'Количество вхождений: {len(result)}')
        for token in result:
            print(f'Найдено: {token[0].form} в предложении {token[1]}')
    else:
        print('Такой команды нет.')
    

Выберите, что вы хотите сделать
	0 - выйти
	1 - прочитать файл
	2 - вывести статистику по частям речи
	3 - искать вхождения по лемме
Выберите, что вы хотите сделать
	0 - выйти
	1 - прочитать файл
	2 - вывести статистику по частям речи
	3 - искать вхождения по лемме
Статистика частей речи
PRON : 6.94%
AUX  : 7.11%
SCONJ: 0.75%
DET  : 16.04%
NOUN : 16.95%
ADP  : 12.0%
PUNCT: 11.78%
VERB : 7.91%
ADV  : 3.53%
ADJ  : 6.32%
CCONJ: 0.41%
PROPN: 8.73%
NUM  : 0.48%
X    : 1.02%
SYM  : 0.0%
INTJ : 0.01%
Выберите, что вы хотите сделать
	0 - выйти
	1 - прочитать файл
	2 - вывести статистику по частям речи
	3 - искать вхождения по лемме
Количество вхождений: 28
Найдено: mort в предложении Quand le cardinal Juan Jesus Posadas Ocampo est-il mort ?

Найдено: mort в предложении Comment Vitas Gerulaitis est-il mort ?

Найдено: mort в предложении Quand Shapour Bakhtiar est-il mort ?

Найдено: mortes в предложении Combien de personnes sont mortes dans l'écrasement d'un avion survenu dans les Everglades en

#### Задача 2 (5 баллов).

Хотим написать программу - базу данных для библиотеки. Нам понадобится класс "книга", который будет содержать автора, название, жанр и количество страниц; а также класс "библиотека", в котором в атрибутах будут сидеть все наши книги. В библиотеку будут поступать запросы: нужно будет выдавать перечень всех книг одного автора, перечень всех книг конкретного жанра, а также проверять, что книга такого автора и с таким названием есть (вам хорошо помогут генераторные выражения для всех этих вещей, а еще можно переопределить магический метод \_\_eq\_\_ у класса "книга" - он должен возвращать bool, а внутри него можно сравнивать атрибуты экземпляров self и other - и проверять наличие объекта класса "книга" в списке). Магический метод \_\_eq\_\_ неявным образом вызывается, когда вы сравниваете два объекта: например, сравниваете две книжки. Он будет неявно вызываться и тогда, когда вы проверяете наличие какого-то объекта в списке таких же объектов с помощью оператора in. 

In [11]:
class Book: #author, title, genre, pages
    def __init__(self, author, title, genre, pages):
        self.author = author
        self.title = title
        self.genre = genre
        self.pages = pages

    def __eq__(self, other):
        return self.author == other.author and self.title == other.title

class Bibliotheque: 
    #all_books
    # demand: all_books by authorX; all_books be genreX, check if bookX of authorX exist
    def __init__(self):
        self.books = []

    def adding_books(self, book):
        self.books.append(book)
        print(f'Книга "{book.title}" добавлена в библиотеку')

    def finding_by_author(self, author):
        books_by_author = []
        for book in self.books:
            if book.author == author:
                books_by_author.append(book)
        return books_by_author
    
    def finding_by_genre(self, genre):
        books_by_genre = []
        for book in self.books:
            if book.genre == genre:
                books_by_genre.append(book)
        return books_by_genre
    
    def book_existence(self, author, title):
        request = Book(author, title, 'XXX', 1)
        return request in self.books
    


In [12]:
#add_bibl
bibl = Bibliotheque()

In [13]:
#adding_books
book1 = Book('Fedden', 'A grammar of Mian', 'Science', 604)
book2 = Book('Boudreault', 'A grammar of Sierra Popoluca', 'Science', 1049)
book3 = Book('Bardugo', 'Shadow and Bone', 'Fantasy', 358)
book4 = Book('Remarque', 'Three comrades', 'Novel', 480)
book5 = Book('Remarque', 'Arch of Triumph', 'War novel', 455)
bibl.adding_books(book1)
bibl.adding_books(book2)
bibl.adding_books(book3)
bibl.adding_books(book4)
bibl.adding_books(book5)
#я хотела добавить книги через цикл фор, но чет не получилось. в итоге выглядит некрасиво(


Книга "A grammar of Mian" добавлена в библиотеку
Книга "A grammar of Sierra Popoluca" добавлена в библиотеку
Книга "Shadow and Bone" добавлена в библиотеку
Книга "Three comrades" добавлена в библиотеку
Книга "Arch of Triumph" добавлена в библиотеку


In [14]:
from termcolor import colored, cprint

while True:
    print('Добро пожаловать в нашу библиотеку! Выберите, что вы хотите сделать\n\t0 - выйти\n\t1 - добавить новую книгу\n\t2 - найти книги по автору\n\t3 - найти книги по жанру\n\t4 - проверить наличие книги в библиотеке')
    user = int(input())
    if user == 0: #exit
        break
    elif user == 1: #adding books
        name, author, genre, pages = input('Введите название книги: '), input('Введите фамилию автора: '), input('Введите жанр книги: '), input('Введите количество страниц: ')
        new_book = Book(author, name, genre, pages)
        bibl.adding_books(new_book)
    elif user == 2: #finding by author
        author = input('Введите фамилию автора: ')
        author_s_books = bibl.finding_by_author(author)
        print(f'Книги автора {author}: ')
        for book in author_s_books:
            print(f'\t{book.title}')
    elif user == 3: #finding by genre
        genre = input('Введите жанр: ')
        books_by_genre = bibl.finding_by_genre(genre)
        print(f'Книги жанра {genre}: ')
        for book in books_by_genre:
            print(f'\t{book.title}')
    elif user == 4: #book existence
        name, author = input('Введите название книги и фамилию автора через запятую').split(', ')
        if bibl.book_existence(author, name):
            print(f'Да, {name} автора {author} есть в библиотеке')
        else:
            print(f'Запрашиваемой книги {name} автора {author} нет в библиотеке')
    else:
        output = colored('Такой команды нет!', 'red')
        print(output)

Добро пожаловать в нашу библиотеку! Выберите, что вы хотите сделать
	0 - выйти
	1 - добавить новую книгу
	2 - найти книги по автору
	3 - найти книги по жанру
	4 - проверить наличие книги в библиотеке
Такой команды нет!
Добро пожаловать в нашу библиотеку! Выберите, что вы хотите сделать
	0 - выйти
	1 - добавить новую книгу
	2 - найти книги по автору
	3 - найти книги по жанру
	4 - проверить наличие книги в библиотеке
Книга "Kings of Scars" добавлена в библиотеку
Добро пожаловать в нашу библиотеку! Выберите, что вы хотите сделать
	0 - выйти
	1 - добавить новую книгу
	2 - найти книги по автору
	3 - найти книги по жанру
	4 - проверить наличие книги в библиотеке
Книги автора Remarque: 
	Three comrades
	Arch of Triumph
Добро пожаловать в нашу библиотеку! Выберите, что вы хотите сделать
	0 - выйти
	1 - добавить новую книгу
	2 - найти книги по автору
	3 - найти книги по жанру
	4 - проверить наличие книги в библиотеке
Книги жанра Science: 
	A grammar of Mian
	A grammar of Sierra Popoluca
Добро п